# Описание задания

В рамках данного задания, студент должен создать и оценить 4 типа рекомендательных систем:
* Non-personalized RS
* Content-based RS
* Item-based collaborative filtering RS
* Hybrid RS

Каждая рекомендательная система - отдельное подзадание. Подзадание считается выполненным, если студент создал рекомендательную систему, которая **лучше (или хуже, но не более чем на 10%)** системы, созданной автором данного блокнота. Системы оцениваются с использованием метрики ``HR@N``, описанной ниже.

# Детальное описание

#### Данные: 
Датасет представлен множеством отзывов к компьютерным играм (объектам) от пользователей Amazon. Каждый отзыв представлен в виде JSON-структуры со следующими полями:
* идентификатор пользователя - reviewerID
* идентификатор объекта - asin
* текст отзыва - reviewText
* рейтинг - overall
* время публикации обзора - unixReviewTime
* другие поля, не использованные автором этого блокнота (смотри полное описание JSON [тут](http://jmcauley.ucsd.edu/data/amazon/))

У каждого объекта есть как минимум 5 отзывов, каждый пользователь написал как минимум 5 отзывов. 
#### Цель: 
Построить рекомендательную систему, предсказывающую объекты, которые пользователь приобретет в ближайшем будущем. Для упрощения мы считаем, что пользователь приобрел объект, если он написал про него отзыв.
#### Подготовка данных:
Данные разделены на тренировочную и тестовую выборки по времени публикации отзывов. Первые 80% данных (более старые) используются как тренировочная выборка, остальные - как тестовая. 

Построение рекомендательной системы (т.е., выбор и тренировка моделей, оптимизация параметров и т.д.) осуществляется **только** с использованием тренировочной выборки. Все параметры, использованные в моделях, **должны быть** получены или объяснены с помощью тренировочных данных. Студент вправе использовать тренировочную выборку как его душе угодно. 

Тестирующая выборка используется **только** для оценки рекомендательной системы.

Для построения рекомендательных моделей также можно использовать JSON-поля из датасета, неиспользованные автором этого блокнота.
#### Оценка качества рекомендательной системы
Цель рекомендательной системы - посоветовать пользователю объекты, которые он захочет приобрести. Для оценки качества такой системы мы воспользуемся метрикой `hit-ratio (HR)`. 

$$
HR = \frac{1}{|U_T|}\sum_{u \in U_T} \mathrm{I}(Rel_u \cap Rec_u)
$$

* $U_T$ - множество пользователей из тестовой выборки
* $Rec_u$ - множество объектов, рекомендованных пользователю $u$ 
* $Rel_u$ - множество объектов, оцененных пользователем $u$ в тестовой выборке
* $\mathrm{I}(Rel_u \cap Rec_u)$ - бинарная функция-индикатор. Функция возвращает 1 если $Rel_u \cap Rec_u \ne \emptyset$, иначе 0

$HR=1$ если для каждого пользователя мы рекомендовали хотя бы один релевантный объект. Так как обычно пользователи просматривают только первые $N$ рекомендаций, мы будем считать метрику $HR@N$, где $N=10$ (т.е. множество $Rec_u$ будет содержать только 10 объектов). 

# Условные обозначения
* `uid` - идентификатор пользователя
* `iid` - идентификатор объекта

# Games RSs

In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from scipy.sparse import random
from scipy.sparse import save_npz

In [4]:
from scipy.sparse import load_npz
from scipy.sparse import vstack

In [8]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "lab_data/Video_Games_5.json"
N = 10

## Анализ данных

In [9]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
        
    return pd.DataFrame(rows, columns=cols)

In [11]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [12]:
print("min-max количество объектов на пользователя:", 
      df.groupby("uid").iid.nunique().min(), df.groupby("uid").iid.nunique().max())
print("min-max количество пользователей на объект:", 
      df.groupby("iid").uid.nunique().min(), df.groupby("iid").uid.nunique().max())

min-max количество объектов на пользователя: 5 773
min-max количество пользователей на объект: 5 802


In [13]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [14]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [15]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [16]:
training_df, test_df = split_df_by_dt(df, p=0.8)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [17]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

Для упрощения тестирования предлагается использовать словарь следующего типа:

```python
recs = {
    uid_1: {
        iid_1: score_11,
        iid_2: score_12,
        ...
    },
    uid_2: {
        iid_1: score_21,
        iid_2: score_22,
        ...
    },
    ...
}
```

где `uid_i` - идентификатор тестового пользователя, `iid_j` - идентификатор рекомендованного объекта, а `score_ij` - предсказанный рейтинг/вес объекта `j` для пользователя `i`.

In [18]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [19]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Базовые классы для рекомендательной системы

In [20]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        from collections import OrderedDict
        return OrderedDict(self.recs[:top])
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}

## Content-based RS

Простая content-based рекомендательная система описывает пользователей и объекты как вектора в некотором N-мерном пространстве фич. Вектор объекта показывает, насколько объект принадлежит к той или иной фиче. Вектор пользователя показывает, насколько пользователь предпочитает ту или иную фичу. Рекомендации строятся путем поиска объектов, чьи вектора похожи на вектор предпочтений пользователя. Предполагается, что чем более похожи вектора пользователя и объекта, тем интереснее этот объект пользователю.

#### `HR@10` для content-based, модели созданной автором блокнота: 0.065

## Item-based collaborative filtering RS

Item-based CF основан на идее, что пользователь предпочтет объекты, похожие на те, что он приобретал ранее. Данные в CF модели представлены матрицей `user x item`, где ячейка матрицы соответствует рейтингу, который пользователь поставил объекту. Вместо рейтингов в матрице могут быть вероятности (т.е. вероятность, что пользователь воспользуется объектом). Для работы модели необходимо построить матрицу `item x item` схожести объектов. Обычно для построения матрицы схожести используется исходная матрица `user x item`. Чтобы уменьшить шумы в матрице схожести, для каждого объекта хранят только $K$ наиболее похожих объектов.

В простейшем случае рекомендации строятся путем нахождения объектов с наибольшим значением предсказанного рейтинга:
$$\hat{r}_{ui} = \frac{\sum_{j \in I_u} r_{uj} * sim(j, i)}{\sum_{j \in I_u} r_{uj}}$$

* $I_u$ - множество объектов, оцененных пользователем
* $sim(j, i)$ - схожесть между объектами $j$ и $i$

Часто из финальных рекомендаций для пользователя $u$ исключаются объекты $I_u$.

In [21]:
# вспомогательные функции, которые могут пригодиться при построении Item-based CF
def nullify_main_diagonal(m):
    positions = range(m.shape[0])
    eye = csr_matrix((np.ones(len(positions)), (positions, positions)), m.shape)
    return m - m.multiply(eye)


def get_topk(matrix, top, axis=1):
    """Converts source matrix to Top-K matrix
    where each row or column contains only top K values

    :param matrix: source matrix
    :param top: number of top items to be stored
    :param axis: 0 - top by column, 1 - top by row
    :return:
    """
    rows = []
    cols = []
    data = []

    if axis == 0:
        matrix = matrix.T.tocsr()

    for row_id, row in enumerate(matrix):
        if top is not None and row.nnz > top:
            top_args = np.argsort(row.data)[-top:]

            rows += [row_id] * top
            cols += row.indices[top_args].tolist()
            data += row.data[top_args].tolist()
        elif row.nnz > 0:
            rows += [row_id] * row.nnz
            cols += row.indices.tolist()
            data += row.data.tolist()

    topk_m = csr_matrix((data, (rows, cols)), (matrix.shape[0], matrix.shape[1]))

    if axis == 0:
        topk_m = topk_m.T.tocsr()

    return topk_m

In [22]:
item_ids = training_df.iid.unique() # уникальные идентификаторы объектов в тренировочной выборке
ids_reviews = {}

# для каждого id из item_ids формируется "суммарное" описание из отзывово нескольких пользователей
for id in item_ids:
    text = training_df[training_df.iid == id].review.str.cat(sep=' ')
    ids_reviews[id] = text

texts = [review for _, review in ids_reviews.items() ] # 

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

Предобработка текстов

In [24]:
import nltk.corpus
from nltk import word_tokenize, wordpunct_tokenize

Формируем словари для препроцессинга

In [25]:
brown = set(nltk.corpus.brown.words())
pr_1 = set(nltk.corpus.product_reviews_1.words())
pr_2 = set(nltk.corpus.product_reviews_2.words())
web = set(nltk.corpus.webtext.words())
nps = set(nltk.corpus.nps_chat.words())
reuters = set(nltk.corpus.reuters.words())
names = set(nltk.corpus.names.words())

In [26]:
dictionary = set()
for dict_ in [brown, pr_1, pr_2, web, nps, reuters, names]:
    dictionary = dictionary.union(dict_)

In [27]:
class TextPreprocessor(object):
    def __init__(self, dictionary):
        self.dictionary = dictionary
    
    def preprocess(self, text):
        dict_ = self.dictionary
        proc_text = " ".join(w for w in wordpunct_tokenize(text) if w.lower() in dict_)
        return proc_text

In [28]:
pp = TextPreprocessor(dictionary).preprocess

In [29]:
my_tfidf = TfidfVectorizer(stop_words='english', preprocessor=pp)
my_tfidf.fit(texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<bound method TextPreprocessor.preprocess of <__main__.TextPreprocessor object at 0x34D15990>>,
        smooth_idf=True, stop_words='english', strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [30]:
del texts

In [31]:
DICT_LEN = len(my_tfidf.vocabulary_)
DICT_LEN

61309

In [32]:
list_of_items = []
id_to_ind = {} #словарь соответсвия id объекта и номера строки в матрице объектов
for id in item_ids:
    list_of_items.append(ids_reviews[id])
    id_to_ind[id] = id_to_ind.setdefault(id, len(id_to_ind))

In [33]:
items_matrix = my_tfidf.transform(list_of_items) #матрица объектов 
del list_of_items

In [34]:
items_matrix 

<10098x61309 sparse matrix of type '<class 'numpy.float64'>'
	with 7542023 stored elements in Compressed Sparse Row format>

In [35]:
items_matrix.nnz/(items_matrix .shape[0]*items_matrix .shape[1])

0.012182270924150295

Строим профили пользователей

In [36]:
from scipy.sparse import vstack

In [37]:
from scipy.sparse import save_npz
from math import ceil

uid_to_ind = {}
ind_to_uid = {}

#формируем разбиение списка пользователей для демпфирования матрицы пользователя
user_rows = len(training_df.uid.unique())
Nsplit = 20 #число разбиений
block = ceil(user_rows/Nsplit)

# создаем список диапазонов индексов
block_list = []
for i in range(Nsplit+1):
    val = i*block
    if val > user_rows:
        val = user_rows
    block_list.append(val)

    
for uid in training_df.uid.unique():
    uid_to_ind[uid] = uid_to_ind.setdefault(uid, len(uid_to_ind))

ind_to_uid = { v: k for k,v in uid_to_ind.items() }

for i in range(Nsplit):
    for k in range(block_list[i], block_list[i+1]):
        uid = ind_to_uid[k]
        user_df = training_df[training_df.uid == uid] #выбираем кусок df, соответствующий пользователю
        user_items = user_df.iid                      #список объектов, оцененных пользователем
        user_ratings = user_df.rating                 #рейтинги пользователя

        user_vect = csr_matrix((1, DICT_LEN))         #инициализируем пустой вектор пользователя
        users_list = []

        for iid, rating in zip(user_items, user_ratings):
            item_vect = items_matrix.getrow(id_to_ind[iid])
            user_vect += item_vect.multiply(rating)
            users_list.append(user_vect)
        try:
            users_matrix = vstack([users_matrix, user_vect])
        except NameError:
            users_matrix = user_vect
        #users_matrix = vstack(users_list)
    save_npz('user_'+str(i), users_matrix)
    del users_matrix

In [40]:
from scipy.sparse import load_npz

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
save_npz('items_matrix', items_matrix)

In [44]:
for i in range(Nsplit):
    users_matrix = load_npz('user_'+str(i)+'.npz')
    similarity_matrix = cosine_similarity(users_matrix, items_matrix, dense_output=False)
    save_npz('similarity_'+str(i))

MemoryError: 

In [46]:
similarity_matrix.shape #users x items

(3284, 2339)

In [130]:
# выбирает соответсвие id объекта - rating, уже приобретенные объекты (exclude keys) исключаются
def select_item(source_matrix, keys_to_index, key_, exclude_keys):
    if key_ in exclude_keys:
        return 0
    else:
        return source_matrix[0, keys_to_index[key_]]

In [162]:
class ContentBasedRecommender_dummy(NonPersRecommender):
    
    def __init__(self, uid_to_ind, iid_to_ind, train_df, sim_matrix):
        super(NonPersRecommender, self).__init__()
        self.uid_to_ind = uid_to_ind
        self.iid_to_ind = iid_to_ind
        self.train_df = train_df
        self.sim_matrix = sim_matrix
        #self.recs = self._prepare_recs(uid_to_ind, iid_to_ind, train_df, sim_matrix)
        
    def get_recs(self, uids_to_recommend, top_k=10):
        self.recs = self._prepare_recs(uids_to_recommend, self.uid_to_ind, self.iid_to_ind, self.train_df, self.sim_matrix, top_k)
        return self.recs
        
    def _prepare_recs(self, uids_to_recommend, uid_to_ind, iid_to_ind, train_df, sim_matrix, top_k):
        out = {}
        for uid in uids_to_recommend:
            already_bought_items = train_df[train_df.uid == uid].iid.values
            sim_row = sim_matrix.getrow(uid_to_ind[uid]).todense()
            item_with_score = { iid: select_item(sim_row, iid_to_ind, iid, already_bought_items) for iid in iid_to_ind.keys() }
            dummy = pd.Series(item_with_score).sort_values(ascending=False)[:top_k].to_dict()
            out[uid] = dummy
        return out

In [163]:
recommender = ContentBasedRecommender_dummy(uid_to_ind, id_to_ind, training_df, similarity_matrix)

In [183]:
uids_to_recommend = test_df.uid.values

In [184]:
uids_to_recommend.shape

(3389,)

In [185]:
recommends = recommender.get_recs(uids_to_recommend)

In [188]:
len(recommends)

987

In [190]:
hit_ratio(recommends, test_dict)

0.12056737588652482

#### `HR@10` для item-based CF модели, созданной автором блокнота: 0.085

### Подcказки
* Определитесь с тем, что вы пытаетесь предсказать (рейтинги, вероятности, ...)
* Оптимальный способ вычисления матрицы схожести выглядит так:
 * Привести строки в матрице `user x item` к единичной длине (выделяет основные предпочтения пользователя)
 * Построить матрицу схожести `item x item`
 * Для каждого объекта оставить только $K$ наиболее похожих объектов
 * Для каждого объекта привести к единичной длине вектор схожести этого объекта (выделяет наиболее схожие объекты)
* Удалили ли вы из рекомендаций объекты, которые пользователь уже оценивал?
* Статья "Item-Based Top-N Recommendation Algorithms", Mukund Deshpande и George Karypis

## Hybrid RS

Гибридная рекомендательная система - это объединение нескольких рекомендательных систем (мы не будем перечислять тут возможные способы гибридизации). Цель гибридизации - воспользоваться сильными сторонами нескольких моделей, чтобы улучшить качество рекомендаций.

В данном задании студент должен создать гибридную систему, состояющую **как минимум** из двух подсистем.

#### `HR@10` для гибридной модели, созданной автором блокнота: 0.096

### Подcказки
* Определите сильные и слабые стороны различных моделей
* Какие из них коррелируют? А какие могут дополнять друг друга?
* Только конечный результат работы системы должен содержать $N$ рекомендаций (промежуточные могут содержать больше)

## P.S.
В коде возможны пасхальные яйца ]:->, если у вас возникли вопросы, не стесняйтесь их задавать.